In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
%matplotlib inline

In [ ]:
data_train = pd.read_csv("../../data/train.csv")
data_test = pd.read_csv("../../data/test.csv")

In [ ]:
print("Train data dims:", data_train.shape)
print("Test data dims:", data_test.shape)

## Graph Exploration

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(range(data_train.shape[0]), np.sort(data_train.y.values))
plt.xlabel('index', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.show()

In [ ]:
var_name = "ID"
plt.figure(figsize=(12,6))
sns.regplot(x=var_name, y='y', data=data_train, scatter_kws={'alpha':0.5, 's':30})
plt.xlabel(var_name, fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title("Distribution of y variable with "+var_name, fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(data_train.y.values, bins=50, kde=False)

In [ ]:
dtype_df = data_train.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df.groupby("Column Type").aggregate('count').reset_index()

In [ ]:
dtype_df.ix[:10,:]

In [ ]:
unique_values_dict = {}
for col in data_train.columns:
    if col not in ["ID", "y", "X0", "X1", "X2", "X3", "X4", "X5", "X6", "X8"]:
        unique_value = str(np.sort(data_train[col].unique()).tolist())
        tlist = unique_values_dict.get(unique_value, [])
        tlist.append(col)
        unique_values_dict[unique_value] = tlist[:]
for unique_val, columns in unique_values_dict.items():
    print("Columns containing the unique values : ",unique_val)
    print(columns)
    print("--------------------------------------------------")

Descartamos las variables que solo tienen un 0, no nos van a ser muy utiles que digamos para predecir Y.

## Attributes

In [ ]:
data = data_train

In [ ]:
del data['ID']

In [ ]:
data.head()

In [ ]:
data = pd.get_dummies(data, prefix=['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6',' X8'])

In [ ]:
data.head()

At this point all variables are either 0 or 1

divide the train dataset into 2 datasets, one part to train, another part to test what to train

In [ ]:
msk = np.random.rand(len(data)) < 0.8
train = data[msk]
test = data[~msk]

In [ ]:
train_x = train[train.columns[1:600]]
train_y = train[train.columns[0:1]]
test_x = test[test.columns[1:600]]
test_y = test[test.columns[0:1]]


X_train = train_x
Y_train = train_y
X_test = test_x
Y_test = test_y

## Simple linear regression

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(train_x, train_y)

In [ ]:
print("Intercept: %.2f" %regr.intercept_)
print("Mean squared error: %.2f" % np.mean((regr.predict(test_x) - test_y) ** 2))
# Explained variance score: 1 is perfect prediction
print('R-squared score: %.2f' % regr.score(test_x, test_y))

In [ ]:
regr.score(test_x, test_y)

In [ ]:
train_y.shape

In [ ]:
(regr.predict(test_x) - test_y)

In [ ]:
import tensorflow as tf

In [ ]:
# Parameters
learning_rate = 0.001
training_epochs = 500
batch_size = 300
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 128 # 2nd layer number of features
n_input = 563 # MNIST data input (img shape: 28*28)
n_classes = 1 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [ ]:
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with sigmoid activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    # Hidden layer with sigmoid activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [ ]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.square(pred-y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
total_len = X_train.shape[0]

# Launch the graph
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(total_len/batch_size)
        # Loop over all batches
        for i in range(total_batch-1):
            batch_x = X_train[i*batch_size:(i+1)*batch_size]
            batch_y = Y_train[i*batch_size:(i+1)*batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c, p = sess.run([optimizer, cost, pred], feed_dict={x: batch_x, y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch

        # sample prediction
        label_value = batch_y
        estimate = p
        err = label_value-estimate
        print "num batch:", total_batch

        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", (epoch+1), "cost=", avg_cost
            print ("[*]----------------------------")
            #for i in xrange(3):
            #    print "label value:" + label_value[i] + "estimated value:" + estimate[i]
            print "[*]============================"

    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: X_test, y: Y_test})